# BGS SV QA

## Load the Spectro Catalog Database

Start by configuring the database, then load exposures, truth, targets, fiberassign, and the redshift catalog.

In [ ]:
import importlib
import desispec.database.redshift
import os
import numpy as np

from argparse                   import Namespace

from desispec.database.redshift import (setup_db, dbSession, load_file, ObsList, Target, Truth, ZCat, FiberAssign,
                                        load_fiberassign, update_truth)

from desitarget.targetmask import (desi_mask, mws_mask, bgs_mask)


importlib.reload(desispec.database.redshift)

In [ ]:
basedir      = '/global/cscratch1/sd/mjwilson/minitest-19.2/'
targetdir    = '/global/cscratch1/sd/mjwilson/minitest-19.2/targets/'
reduxdir     = '/global/cscratch1/sd/mjwilson/minitest-19.2/spectro/redux/mini/'
expfile      = '/global/cscratch1/sd/mjwilson/minitest-19.2/survey/exposures.fits'
fibassigndir = '/global/cscratch1/sd/mjwilson/minitest-19.2/fiberassign/'

options      = Namespace(overwrite=True, dbfile=os.path.join(basedir, 'minitest.db'), hostname=None, maxrows=2000,
                         chunksize=50000, schema=None, username=None, verbose=False, datapath=basedir)

# We'll be using a SQLite database, ignore the return value of setup_db.
postgresql = setup_db(options)

print('--- Loading exposure list')
load_file(expfile, ObsList, hdu='EXPOSURES', expand={'PASS': 'passnum'})

print('--- Loading truth tables')
load_file(os.path.join(targetdir, 'truth.fits'), Truth, hdu='TRUTH')

for h in ('BGS', 'ELG', 'LRG', 'QSO', 'STAR', 'WD'):
    update_truth(os.path.join(targetdir, 'truth.fits'), 'TRUTH_' + h)

print('--- Loading targets')
load_file(os.path.join(targetdir, 'targets.fits'), Target, hdu='TARGETS',
          expand={'DCHISQ': ('dchisq_psf', 'dchisq_rex', 'dchisq_dev', 'dchisq_exp', 'dchisq_comp',)})

print('--- loading redshift catalog')
load_file(os.path.join(reduxdir, 'zcatalog-mini.fits'), ZCat, hdu='ZCATALOG',
          expand={'COEFF': ('coeff_0', 'coeff_1', 'coeff_2', 'coeff_3', 'coeff_4',
                            'coeff_5', 'coeff_6', 'coeff_7', 'coeff_8', 'coeff_9',)},
          rowfilter=lambda x: ((x['TARGETID'] != 0) & (x['TARGETID'] != -1)))

print('--- loading fiber assignments')
load_fiberassign(fibassigndir)

print('--- done')

### Demonstrate the SQLAlchemy objects

In [ ]:
tt = dbSession.query(Truth, Target).filter(Truth.targetid == Target.targetid).all()

In [ ]:
zz = dbSession.query(Truth, ZCat).filter(Truth.targetid == ZCat.targetid).all()

In [ ]:
ff = dbSession.query(Truth, FiberAssign).filter(Truth.targetid == FiberAssign.targetid).all()

In [ ]:
# How many actual exposures are there with the Moon up?
q = dbSession.query(ObsList.expid, ObsList.moonsep, ObsList.moonalt, ObsList.moonfrac).filter(ObsList.moonalt > 0).all()

In [ ]:
'''
q = dbSession.query(ZCat.targetid, Target.desi_target, Target.bgs_target, Target.mws_target, ObsList.expid)\
             .filter(ZCat.targetid      == FiberAssign.targetid)\
             .filter(ZCat.targetid      == Target.targetid)\
             .filter(FiberAssign.tileid == ObsList.tileid)\
             .filter(ObsList.expid.in_(expid))\
             .order_by(ZCat.targetid, ObsList.expid).all()

targetid, desi_target, bgs_target, mws_target, expid = zip(*q)
'''

In [ ]:
'''
print(sum(['ELG'     in desi_mask.names(t) for t in desi_target]))
print(sum(['BGS_ANY' in desi_mask.names(t) for t in desi_target]))
print(sum(['BGS_ANY' in  bgs_mask.names(t) for t in  bgs_target]))
'''

In [ ]:
#.filter(ObsList.expid.in_(expid))\
q = dbSession.query(ZCat.targetid, Truth.truez, ZCat.z, ZCat.zwarn, Target.fiberflux_r, ((ZCat.z - Truth.truez)/(1.0 + Truth.truez)).label('dz'))\
                   .filter(Truth.targetid     == ZCat.targetid)\
                   .filter(Target.targetid    == ZCat.targetid)\
                   .filter(ZCat.targetid      == FiberAssign.targetid)\
                   .filter(FiberAssign.tileid == ObsList.tileid)\
                   .filter(Target.desi_target.op('&')(desi_mask.BGS_ANY) != 0)\
                   .all()

_, truez, z, zwarn, fiberflux, dz = zip(*q)

z     = np.array(z)
truez = np.array(truez)
zwarn = np.array(zwarn)

##  binned fiber flux. 
bins     = np.arange(0.0, 5., 1.)
nbins    = len(bins)

bnd_ffux = np.digitize(fiberflux, bins, right=False)

In [ ]:
q[0].targetid

In [ ]:
# What warnings were there?
print(np.unique(zwarn, return_counts=True))

In [ ]:
import matplotlib.pyplot as plt


# Observed redshift versus true redshift.
fig, axes = plt.subplots(3, nbins, figsize=(14, 8), dpi=100, sharey = True, sharex=True)

for i, _zwarn in enumerate([0, 4, 36]):
 for j, lolim in enumerate(bins):
  axes[i][j].plot(truez[(zwarn == _zwarn) & (bnd_ffux == j)], z[(zwarn == _zwarn) & (bnd_ffux == j)], 'ko', label='', markersize=1)

  axes[i][j].set_xlim(0.0, 1.9)
  axes[i][j].set_ylim(0.0, 1.9)
    
  axes[i][j].legend(loc=2, frameon=False, title='(%d, %.1lf)' % (_zwarn, bins[j]))
    
axes[0][0].set_ylabel(r'$\hat z$')
axes[1][0].set_ylabel(r'$\hat z$')
axes[2][0].set_ylabel(r'$\hat z$')

axes[2][0].set_xlabel(r'$z$')
axes[2][1].set_xlabel(r'$z$')
axes[2][2].set_xlabel(r'$z$')    
axes[2][3].set_xlabel(r'$z$')    
axes[2][4].set_xlabel(r'$z$')    

fig.suptitle('Fiber flux')

In [ ]:
# Print the table columns and their types.
[(c.name, c.type) for c in ObsList.__table__.columns]

In [ ]:
# Print the table columns and their types.
[(c.name, c.type) for c in ZCat.__table__.columns]

In [ ]:
# Print the table columns and their types.
[(c.name, c.type) for c in Target.__table__.columns]

In [ ]:
# Print the table columns and their types.
[(c.name, c.type) for c in FiberAssign.__table__.columns]

In [ ]:
#type(tt[0][0])

In [ ]:
#dir(tt[0][0])

In [ ]:
tt[0][0]

In [ ]:
type(tt[0][1])

In [ ]:
#dir(tt[0][1])

In [ ]:
type(zz[0][0])

In [ ]:
#dir(zz[0][0])

In [ ]:
type(zz[0][1])

In [ ]:
#dir(zz[0][1])

In [ ]:
import numpy as np

np.unique(zz[0][1].zwarn, return_counts=True)

In [ ]:
type(ff[0][0])

In [ ]:
#dir(ff[0][0])

In [ ]:
type(ff[0][1])

In [ ]:
#dir(ff[0][1])

# QA

## Initialize QA output directory

In [ ]:
import desispec

# New dir to define and make
qaprod_dir = desispec.io.qaprod_root()
os.makedirs(qaprod_dir, exist_ok=True)
qaprod_dir

In [ ]:
#- Helper function for timing QA commands
def time_command(cmd, logfile):
    t0 = time.time()
    print('{} RUNNING {}'.format(time.asctime(), cmd))
    err = subprocess.call(cmd.split(), stdout=logfile, stderr=logfile)
    if err != 0:
        print('FAILED: {}'.format(cmd))

    dt = time.time() - t0
    if dt < 60:
        print('"{}" took {:.1f} seconds'.format(cmd, time.time()-t0))
    else:
        print('"{}" took {:.1f} minutes'.format(cmd, dt/60))
    return err

### QA with the Truth

The following QA uses the input truth table.  

In [ ]:
qat_logname = os.path.join(qaprod_dir, 'qa_truth.log')
print('logging truth-based QA to {}'.format(qat_logname))

In [ ]:
import time, glob, subprocess

qa0_time = time.time()

with open(qat_logname, 'w') as logfile:
    if len(glob.glob(qaprod_dir+'/QA_s2n_*')) == 10:
        print("S/N figures already exist")
        
    else:
        # S/N (~7min)
        cmd = "desi_qa_s2n --qaprod_dir={:s}".format(qaprod_dir)
        time_command(cmd, logfile)
    
    # zfind (~2min)
    if (len(glob.glob(qaprod_dir+'/QA_zfind_*')) == 6) and os.path.exists(qaprod_dir+'/QA_dzsumm.png'):
        print("zfind figures already exist")

    else:
        cmd = "desi_qa_zfind --yaml_file={:s}/dzsumm_stats.yaml --qaprod_dir={:s}".format(qaprod_dir, qaprod_dir) 
        time_command(cmd, logfile)
    
# Time me
print("Done with QA with truth at {}".format(time.asctime()))

qa_truth_time = time.time() - qa0_time

print("That took {:.1f} minutes".format(qa_truth_time/60))

### Check

In [ ]:
assert len(glob.glob(qaprod_dir+'/QA_s2n_*')) == 10
assert len(glob.glob(qaprod_dir+'/QA_zfind_*')) == 6
assert os.path.exists(qaprod_dir+'/QA_dzsumm.png')

### Show

In [ ]:
from IPython.display import Image, display

#Image(filename=qaprod_dir+'/QA_dzsumm.png') 

In [ ]:
Image(filename=qaprod_dir+'/QA_zfind_ELG.png', width=500)

In [ ]:
Image(filename=qaprod_dir+'/QA_zfind_LRG.png', width=500)

In [ ]:
display(
    Image(filename=qaprod_dir+'/QA_zfind_QSO_T.png', width=500),
    Image(filename=qaprod_dir+'/QA_zfind_QSO_L.png', width=500),
)

In [ ]:
display(
    Image(filename=qaprod_dir+'/QA_zfind_MWS.png', width=500),
    Image(filename=qaprod_dir+'/QA_zfind_BGS.png', width=500),
)

In [ ]:
print("Timing checkpoint at {}".format(time.asctime()))
tmp_time = time.time() - notebook_start_time
print("{:.1f} minutes so far".format(tmp_time/60))